# レッスン 05 GPT-2とNanoGPT

GPT-2は2019年に発表された際、OpenAIは重要な発見を示しました。Transformerベースの言語モデルを前例のない規模まで拡大すると——GPT-1の1億1700万パラメータから15億パラメータへと増加させ、より大規模で高品質なウェブテキストで訓練すると、モデルは驚くべき能力を示すようになりました。長い一貫性のあるテキストを生成できるだけでなく、より重要なことに、次の単語を予測するという単純な訓練目標だけで、モデルは明示的に訓練されていない様々なタスク——翻訳、質問応答、要約、さらには簡単な算術計算まで——を実行できることを証明しました。この「言語モデルは教師なしマルチタスク学習器である」という理念は、NLPモデルの訓練に対する私たちの理解を根本的に変えました。

GPT-2は技術的には純粋なDecoderアーキテクチャのTransformerを採用し、元のTransformerのエンコーダー部分を取り除いて、モデル構造をより簡潔で統一的なものにしました。12層から48層のTransformerブロックを使用し、各ブロックにはマルチヘッド自己注意機構とフィードフォワードニューラルネットワークが含まれ、層正規化と残差接続が組み合わされています。この一見シンプルなアーキテクチャは、慎重に選別された40GBのWebTextデータセットで訓練することで、強力な言語理解と生成能力を示しました。

nanoGPTはAndrej Karpathyが作成した教育プロジェクトで、最小限のコードでGPT-2のコア機能を再現することを目的としています。実装全体は約300行のPythonコードのみですが、GPTアーキテクチャの重要なコンポーネント——因果的自己注意機構、位置エンコーディング、層正規化、そして自己回帰生成——を完全に実装しています。自分のノートパソコンで小規模なGPTを訓練できます。例えば、シェイクスピアの作品集でシェイクスピア風のテキストを生成できるモデルを訓練したり、簡単な数学データセットで加算ができるモデルを訓練したりできます。この極めてシンプルな実装により、私たちは各行のコードの役割を真に理解し、エンジニアリングの最適化や複雑な訓練技術に隠されることなく、Transformerの本質を明確に見ることができます。

## GPT-2について

![GPT2Scaling](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/scaling_laws.png?raw=true)

この図はGPT-2論文の中核的な発見を示しています。4つのグラフはそれぞれ異なるNLPタスクにおいて、モデルサイズ（117Mから1.5Bパラメータ）が増加するにつれてゼロショット性能がどのように向上するかを示しています。

最も重要な観察は、すべてのタスクで一貫した傾向が見られることです——パラメータ数が増えるにつれて、性能が対数的に向上しています。これは「スケーリング則」の初期の証拠であり、後のGPT-3やそれ以降の大規模言語モデル開発の基礎となりました。

![GPTzeroshot](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/zero-shot.png?raw=true)

この本質的に示しているのは、十分に大きな言語モデルは、次の単語を予測するという単純な目標で訓練されただけで、明示的な指示なしに多様なタスクを実行する能力を獲得するということです。

これが「言語モデルは教師なしマルチタスク学習器」というGPT-2の中心的な主張です。

nanoGPTの文脈では、この同じアーキテクチャ——ただしはるかに小さいスケールで——がどのように実装されているかを学生に示すことができます。

スケールは異なっても、基本的な原理は同じです：自己回帰的な次トークン予測を通じて、モデルは言語の構造とパターンを学習します。

![GPTzeroshot](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/key.png?raw=true)

この部分はGPT-2論文の核心的な洞察を説明しています。「言語モデルは教師なしマルチタスク学習器である」というタイトルが示すように、GPT-2の革新的な発見は、言語モデルが明示的な教師データなしに多様なタスクを学習できることです。

論文は重要な観察を述べています：McCann et al. (2018)のような従来のマルチタスク学習では、どの記号を予測すべきかを明示的に指定する必要がありました。

しかしGPT-2は、教師なし目的関数（次の単語予測）と教師あり目的関数の違いは、実は評価する系列の部分集合だけであることを指摘します。

この洞察により、十分に大きな言語モデルは、教師なし学習を通じてマルチタスク学習を実現できる可能性が開かれました。

表1の翻訳例は特に興味深いです。WebTextの訓練データには、自然に発生する翻訳のデモンストレーションが含まれていました：

「Je ne suis pas un imbecile [I'm not a fool].」のように、フランス語の後に英訳が括弧内に示される例。

「Mentez mentez, il en restera toujours quelque chose」という文に「Lie lie something will always remain」という翻訳が続く例。

映画のタイトル「Brevet Sans Garantie Du Gouvernement」が「Patented without government warranty」と翻訳される例。

これらの例が示すのは、GPT-2が訓練中にこのような文脈内の翻訳パターンを学習し、「translate to French:」や「As-tu aller au cinéma?, or Did you go to the movies?」のような形式を理解できるようになったということです。


**「教師あり目的関数は教師なし目的関数と同じだが、系列の部分集合でのみ評価される。したがって、教師なし目的関数の大域的最小値は、教師あり目的関数の大域的最小値でもある」**


従来の教師あり学習では、「質問→答え」のペアを用意し、モデルに答えの部分だけを予測させます。一方、言語モデルは系列全体のすべてのトークンを予測します。GPT-2の著者たちが気づいたのは、これらは実は同じ最適化問題であるということです——違いは損失を計算する場所だけです。

具体例で説明すると：
- 教師あり翻訳：「Bonjour → Hello」（Helloの部分だけで損失を計算）
- 言語モデル：「Translate to English: Bonjour means Hello」（全トークンで損失を計算）

もし言語モデルが完璧に次の単語を予測できるなら、それは必然的に「Bonjour」の後に「Hello」を生成することも学習しているはずです。

つまり、**より一般的な問題（全系列の予測）を解くことで、特定の問題（翻訳、QA、要約など）も同時に解いている**のです。

この理論的保証があるからこそ、GPT-2やその後継モデルは、ただひたすら次の単語を予測する訓練をするだけで、驚くほど多様なタスクを実行できるようになります。nanoGPTも同じ原理で動作します——シンプルな自己回帰的予測が、実は普遍的な問題解決能力につながるのです。





## nanoGPT

GPT-2が証明した「次単語予測だけでマルチタスク学習が可能」という理論を、実際に手を動かして確認してみましょう。

nanoGPTを使えば、ノートパソコンで本物のGPTを訓練できます。環境構築も簡単で、PyTorchさえあれば動きます。

In [ ]:
!pip install torch numpy transformers datasets tiktoken wandb tqdm

まず必要なパッケージをインストールします。nanoGPTの実行に必要な最小限のライブラリセットです：

```bash
pip install torch numpy transformers datasets tiktoken wandb tqdm
```

**各パッケージの役割：**
- `torch`: PyTorchフレームワーク。ニューラルネットワークの構築と訓練の基盤
- `numpy`: 数値計算。データの前処理に使用
- `transformers`: HuggingFaceのライブラリ。事前訓練済みモデルのダウンロードに使用
- `datasets`: HuggingFaceのデータセットライブラリ
- `tiktoken`: OpenAIのトークナイザー。テキストをトークンに分割
- `wandb`: Weights & Biases。訓練の可視化（オプション、スキップ可）
- `tqdm`: プログレスバーの表示

インストールには1-2分程度かかります。GPUがある場合は、CUDAバージョンに対応したPyTorchがインストールされているか確認してください。

### 作業ディレクトリの設定

このノートブックはnanoGPTプロジェクトと同じ階層に配置されているため、まずnanoGPTディレクトリに移動します：

In [ ]:
import os
os.chdir('nanoGPT')

シェイクスピアデータセットを訓練用に準備します。

文字レベルのトークン化は最もシンプルな方法です。GPT-2は本来BPE（Byte Pair Encoding）を使用しますが、教育目的では文字レベルの方が理解しやすく、訓練も速いです。

準備が完了すると、data/shakespeare_char/ディレクトリに訓練データが保存されます。

In [ ]:
!python data/shakespeare_char/prepare.py

### GPUを使った訓練の実行

GPUが利用可能な場合、小規模なGPTモデルを高速に訓練できます。用意されている設定ファイル config/train_shakespeare_char.py を使用します：

In [ ]:
!python train.py config/train_shakespeare_char.py